In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import gridplot, column
from bokeh.io import output_notebook, export_png
from bokeh.models import ContinuousColorMapper, Ticker, LogTicker, ColorBar , DataRange1d
from bokeh.models import LinearColorMapper, BasicTicker, PrintfTickFormatter, ColumnDataSource, LabelSet, Label
from bokeh.transform import transform
from bokeh.palettes import viridis
output_notebook()
colors = ['#4575b4','#91bfdb','#e0f3f8','#ffffbf','#fee090','#fc8d59','#d73027'] 
colors1 = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628', '#f781bf', '#4575b4','#91bfdb','#e0f3f8','#ffffbf','#fee090','#fc8d59','#d73027' ]

In [ ]:
num_real = 1000
start = '2020-03-01'
path2data = '~/Dropbox/EnKF_seir/run/'

### Load simulation files

In [ ]:
col_names = ['time', 'ave', 'std'] + [str(x) for x in range(1, num_real+1)]

In [ ]:
# Load dead
dead_0 = pd.read_csv(os.path.join(path2data, 'dead_0.dat'), sep='\s+', skiprows=52, header=0, dtype=float, names=col_names)
dead_1 = pd.read_csv(os.path.join(path2data,'dead_1.dat'), sep='\s+', skiprows=52, header=0, dtype=float, names=col_names)
dead_0.set_index('time', inplace=True)
dead_1.set_index('time', inplace=True)
dead_0.index = pd.date_range(start = start, periods = dead_0.shape[0], freq='d')
dead_1.index = pd.date_range(start = start, periods = dead_0.shape[0], freq='d')

In [ ]:
# Load hospitalized
hosp_0 = pd.read_csv(os.path.join(path2data,'hosp_0.dat'), sep='\s+', skiprows=52, header=0, dtype=float, names=col_names)
hosp_1 = pd.read_csv(os.path.join(path2data,'hosp_1.dat'), sep='\s+', skiprows=52, header=0, dtype=float, names=col_names)
hosp_0.set_index('time', inplace=True)
hosp_1.set_index('time', inplace=True)
hosp_0.index = pd.date_range(start = start, periods = dead_0.shape[0], freq='d')
hosp_1.index = pd.date_range(start = start, periods = dead_0.shape[0], freq='d')

In [ ]:
# Load exposed
expos_0 = pd.read_csv(os.path.join(path2data,'expos_0.dat'), sep='\s+', skiprows=52, header=0, dtype=float, names=col_names)
expos_1 = pd.read_csv(os.path.join(path2data,'expos_1.dat'), sep='\s+', skiprows=52, header=0, dtype=float, names=col_names)
expos_0.set_index('time', inplace=True)
expos_1.set_index('time', inplace=True)
expos_0.index = pd.date_range(start = start, periods = expos_0.shape[0], freq='d')
expos_1.index = pd.date_range(start = start, periods = expos_0.shape[0], freq='d')

In [ ]:
# Load infectuous
infec_0 = pd.read_csv(os.path.join(path2data,'infec_0.dat'), sep='\s+', skiprows=52, header=0, dtype=float, names=col_names)
infec_1 = pd.read_csv(os.path.join(path2data,'infec_1.dat'), sep='\s+', skiprows=52, header=0, dtype=float, names=col_names)
infec_0.set_index('time', inplace=True)
infec_1.set_index('time', inplace=True)
infec_0.index = pd.date_range(start = start, periods = infec_0.shape[0], freq='d')
infec_1.index = pd.date_range(start = start, periods = infec_0.shape[0], freq='d')

In [ ]:
# Load suseptible
susc_0 = pd.read_csv(os.path.join(path2data,'susc_0.dat'), sep='\s+', skiprows=52, header=0, dtype=float, names=col_names)
susc_1 = pd.read_csv(os.path.join(path2data,'susc_1.dat'), sep='\s+', skiprows=52, header=0, dtype=float, names=col_names)
susc_0.set_index('time', inplace=True)
susc_1.set_index('time', inplace=True)
susc_0.index = pd.date_range(start = start, periods = susc_0.shape[0], freq='d')
susc_1.index = pd.date_range(start = start, periods = susc_0.shape[0], freq='d')

In [ ]:
# Load suseptible
recov_0 = pd.read_csv(os.path.join(path2data,'recov_0.dat'), sep='\s+', skiprows=52, header=0, dtype=float, names=col_names)
recov_1 = pd.read_csv(os.path.join(path2data,'recov_1.dat'), sep='\s+', skiprows=52, header=0, dtype=float, names=col_names)
recov_0.set_index('time', inplace=True)
recov_1.set_index('time', inplace=True)
recov_0.index = pd.date_range(start = start, periods = recov_0.shape[0], freq='d')
recov_1.index = pd.date_range(start = start, periods = recov_0.shape[0], freq='d')

In [ ]:
# Load total amount of cases
case_0 = pd.read_csv(os.path.join(path2data,'case_0.dat'), sep='\s+', skiprows=52, header=0, dtype=float, names=col_names)
case_1 = pd.read_csv(os.path.join(path2data,'case_1.dat'), sep='\s+', skiprows=52, header=0, dtype=float, names=col_names)
case_0.set_index('time', inplace=True)
case_1.set_index('time', inplace=True)
case_0.index = pd.date_range(start = start, periods = case_0.shape[0], freq='d')
case_1.index = pd.date_range(start = start, periods = case_0.shape[0], freq='d')

### Load observation file

In [ ]:
obs_dead = pd.read_csv(os.path.join(path2data, 'obsD.dat'), sep='\s+', skiprows=2, header=0, dtype=float, usecols=[0,1,2,3], names=['idx', 'time', 'obs', 'std'])
obs_dead.set_index('time', inplace=True)
obs_dead.index = pd.date_range(start = dead_1.index[0] + timedelta(days=obs_dead.index[0]), periods = obs_dead.shape[0], freq='d')

In [ ]:
obs_hosp = pd.read_csv(os.path.join(path2data, 'obsH.dat'), sep='\s+', skiprows=2, header=0, dtype=float, usecols=[0,1,2,3], names=['idx', 'time', 'obs', 'std'])
obs_hosp.set_index('time', inplace=True)
obs_hosp.index = pd.date_range(start = hosp_1.index[0] + timedelta(days=obs_hosp.index[0]), periods = obs_hosp.shape[0], freq='d')

### Define colors

In [ ]:
# hosp and deaths
color_dead = '#4daf4a'
color_dead_avg = '#388036'
color_hosp = '#9843a3'
color_hosp_avg = '#763f7e'
# exposed and infected
color_expos = 'darkorange'
color_expos_avg = '#ca871a'
color_infec = '#4daf4a'
color_infec_avg = '#388036'
# recovered and cases
color_recov = '#377eb8'
color_recov_avg = '#377eb8'
color_case = '#feb24c'
color_case_avg = '#fc4e2a'
# susceptible
color_susc = 'darkcyan'
color_susc_avg = '#016b6b'
# Observations
color_obs_dead = 'red'
color_obs_hosp = 'red'

### Hospitalized and dead

In [ ]:
p1 = figure(x_axis_type="datetime", title='Forecast for hospitalized and dead', y_range= DataRange1d(start=0), plot_height=500, plot_width=960)

#Plot realizations
for real in (str(x) for x in range(1, 100)): #num_real+1 (to plot all realizations)
    p1.line(dead_1.index, dead_1[real], color=color_dead, line_width=0.5, alpha = 0.3, legend_label='Dead_1_realizations')
    p1.line(hosp_1.index, hosp_1[real], color=color_hosp, line_width=0.5, alpha = 0.3, legend_label='Hospitalized_1_realizations')

#Plot simulations
p1.varea(x= dead_1.index, y1=dead_1['ave']-2*dead_1['std'], y2=dead_1['ave']+2*dead_1['std'], color=color_dead, alpha=0.2, legend_label='Dead_1_std')
p1.line(dead_1.index, dead_1['ave'], color=color_dead_avg, line_width=3, legend_label='Dead_1_avg')
p1.varea(x= hosp_1.index, y1=hosp_1['ave']-2*hosp_1['std'], y2=hosp_1['ave']+2*hosp_1['std'], color=color_hosp, alpha=0.2, legend_label='Hospitalized_1_std')
p1.line(hosp_1.index, hosp_1['ave'], color=color_hosp_avg, line_width=3, legend_label='Hospitalized_1_avg')

#Plot observations
p1.circle(obs_dead.index, obs_dead['obs'], color=color_obs_dead, size=6, legend_label='Observations dead')
p1.triangle(obs_hosp.index, obs_hosp['obs'], color=color_obs_hosp, size=6, legend_label='Observations hospitalized')

err_xd = []
err_yd = []
err_xh = []
err_yh = []

for xd, yd, yerrd in zip(obs_dead.index, obs_dead['obs'], obs_dead['std']):
    err_xd.append((xd, xd))
    err_yd.append((yd - 2*yerrd , yd + 2*yerrd))
for xh, yh, yerrh in zip(obs_hosp.index, obs_hosp['obs'], obs_hosp['std']):
    err_xh.append((xh, xh))
    err_yh.append((yh - 2*yerrh , yh + 2*yerrh))
    
p1.multi_line(err_xd, err_yd, color=color_obs_dead, legend_label='Observations dead')
p1.multi_line(err_xh, err_yh, color=color_obs_dead, legend_label='Observations hospitalized')

p1.grid.grid_line_alpha=0.9
p1.xaxis.axis_label = 'Time'
p1.yaxis.axis_label = 'Number of people'
p1.legend.click_policy='hide'
p1.legend.location = "bottom_right"
#show(p1)
#export_png(p1,filename='bokehDH.png')

### Exposed and infected

In [ ]:
p2 = figure(x_axis_type="datetime", title='Forecast for exposed and infected', x_range=p1.x_range, y_range= DataRange1d(start=0), plot_height=500, plot_width=960)

#Plot realizations
for real in (str(x) for x in range(1, 100)): #num_real+1 (to plot all realizations)
    p2.line(expos_1.index, expos_1[real], color=color_expos, line_width=0.5, alpha = 0.3, legend_label='Exposed_1_realizations')
    p2.line(infec_1.index, infec_1[real], color=color_infec, line_width=0.5, alpha = 0.3, legend_label='Infected_1_realizations')

#Plot simulations
p2.varea(x= expos_1.index, y1=expos_1['ave']-2*expos_1['std'], y2=expos_1['ave']+2*expos_1['std'], color=color_expos, alpha=0.2, legend_label='Exposed_1_std')
p2.line(expos_1.index, expos_1['ave'], color=color_expos_avg, line_width=3, legend_label='Exposed_1_avg')
p2.varea(x= infec_1.index, y1=infec_1['ave']-2*infec_1['std'], y2=infec_1['ave']+2*infec_1['std'], color=color_infec, alpha=0.2, legend_label='Infected_1_std')
p2.line(infec_1.index, infec_1['ave'], color=color_infec_avg, line_width=3, legend_label='Infected_1_avg')

p2.grid.grid_line_alpha=0.9
p2.xaxis.axis_label = 'Time'
p2.yaxis.axis_label = 'Number of people'
p2.legend.click_policy='hide'
p2.legend.location = "top_right"
#show(p2)

### Recovered and total amount of cases

In [ ]:
p3 = figure(x_axis_type="datetime", title='Forecast for recovered and total amount of cases', x_range=p1.x_range, y_range= DataRange1d(start=0), plot_height=500, plot_width=960)

#Plot realizations
for real in (str(x) for x in range(1, 100)): #num_real+1 (to plot all realizations)
    p3.line(recov_1.index, recov_1[real], color=color_recov, line_width=0.5, alpha = 0.6, legend_label='Recovered_1_realizations')
    p3.line(case_1.index, case_1[real], color=color_case, line_width=0.5, alpha = 0.6, legend_label='Cases_1_realizations')

#Plot simulations
p3.varea(x= recov_1.index, y1=recov_1['ave']-2*recov_1['std'], y2=recov_1['ave']+2*recov_1['std'], color=color_recov, alpha=0.2, legend_label='Recovered_1_std')
p3.line(recov_1.index, recov_1['ave'], color=color_recov_avg, line_width=3, legend_label='Recovered_1_avg')
p3.varea(x= case_1.index, y1=case_1['ave']-2*case_1['std'], y2=case_1['ave']+2*case_1['std'], color=color_case, alpha=0.2, legend_label='Cases_1_std')
p3.line(case_1.index, case_1['ave'], color=color_case_avg, line_width=3, legend_label='Cases_1_avg')

p3.grid.grid_line_alpha=0.9
p3.xaxis.axis_label = 'Time'
p3.yaxis.axis_label = 'Number of people'
p3.legend.click_policy='hide'
p3.legend.location = "top_left"
#show(p3)

### Susceptible

In [ ]:
p4 = figure(x_axis_type="datetime", title='Forecast for susceptible', x_range=p1.x_range,y_range= DataRange1d(start=0), plot_height=500, plot_width=960)

#Plot realizations
for real in (str(x) for x in range(1, 100)): #num_real+1 (to plot all realizations)
    p4.line(susc_1.index, susc_1[real], color=color_susc, line_width=0.5, alpha = 0.3, legend_label='Susceptible_1_realizations')

#Plot simulations
p4.varea(x= susc_1.index, y1=susc_1['ave']-2*susc_1['std'], y2=susc_1['ave']+2*susc_1['std'], color=color_susc, alpha=0.2, legend_label='Susceptible_1_std')
p4.line(susc_1.index, susc_1['ave'], color=color_susc_avg, line_width=3, legend_label='Susceptible_1_avg')

p4.grid.grid_line_alpha=0.9
p4.xaxis.axis_label = 'Time'
p4.yaxis.axis_label = 'Number of people'
p4.legend.click_policy='hide'
p4.legend.location = "top_right"
#show(p4)

In [ ]:
p = column([p1, p2, p3, p4])
show(p)